# Dihedral & Crankshaft Monte Carlo example

In this example, the 1CTF protein sequence will be used in a Monte Carlo simulation. For this example, the protein dihedrals will be randomly rotated by bpth dihedral and crankshaft movements. Crankshaft movements rotate all atoms in between two alpha carbons by a set angle. The new conformation will then be evaluated by calculating the energy according to the Amber forcefield. Secondary structure blocks defined in the input files will not be tempered with, remaining as unified blocks, and the step_size for new movements will de adjusted at every step using a **CallbackObject**.

In [1]:
using ProtoSyn;
using LinearAlgebra;
using Printf;

## 1. Load a state

The first step is to load a starting state from a structural file. ProtoSyn enables loading of coordinates from several standardized file extensions, such as PDB. In this example, the `1ctf.pdb` file was produced using an auxiliar tool that places the amicoacid sequence in a straight configuration, with all dihedrals as 180º. This will be the simulation starting point.

In [2]:
state = Common.load_from_pdb("data/1ctf.pdb");
state.energy = Forcefield.Amber.Energy();

## 2. (Optional) Apply secondary structure

For this particular example, we want to rotate the `coil` aminoacids only, leaving the native secondary structure untamperd with. For this, a topology file must be loaded, containing information about the dihedrals and residues in the molecule. This file also contains information about the secondary structure of each aminoacid. After loading the topology, each PHI and PSI dihedral in the molecule will undergo rotation to apply alpha helix and beta sheets angles. Altough not necessary, a filtering system is exemplified here, easily identifying only the backbone dihedrals.

In [3]:
mc_topology = Aux.read_JSON("data/1ctf_mc_top.json");
dihedrals, residues = Common.load_topology(mc_topology);
bb_dihedrals = filter(x -> x.dtype < Common.omega, dihedrals);
Common.apply_initial_conf!(state, bb_dihedrals)

## 3. (Optional) Fix the damn Prolines

Let's be honest here: nobody likes the Prolines. They are weird and ugly. This function will most likely be fixed in later versions of the ProtoSyn library.

In [4]:
Common.fix_proline(state, dihedrals);

## 4. Define the Sampler

For this example, a combination of two **Mutators** will be employed in a single **Sampler**. Moreover, notice that the unnamed `angle_sampler` uses the `mutator.step_size` as input to adjust the degree of change in the conformation.

In [6]:
bb_nb_dihedrals = filter(x -> x.dtype < Common.omega && Int(x.residue.ss) < 1, dihedrals);
dihedral_mutator = Mutators.Dihedral.DihedralMutator(bb_nb_dihedrals, () -> (rand() * 2 - 1 * dihedral_mutator.step_size), 0.01, 0.01);
crankshaft_mutator = Mutators.Crankshaft.CrankshaftMutator(bb_nb_dihedrals, () -> (rand() * 2 - 1 * crankshaft_mutator.step_size), 0.005, 0.01);
function my_sampler!(st::Common.State)
    Mutators.Dihedral.run!(st, dihedral_mutator)
    Mutators.Crankshaft.run!(st, crankshaft_mutator)
end

my_sampler! (generic function with 1 method)

## 5. Define the Evaluator

In a Monte Carlo simulation, the new conformation generated by the Sampler needs to be evaluated so it can be accepted, or not. In this example, the total system energy will be calculated according to the Amber forcefield. For this, ProtoSyn needs to have access to information regarding the bonds, angles, dihedrals and non-bonded interactions of the molecule, avaliable by loading a topology file.

In [7]:
topology = Forcefield.Amber.load_from_json("data/1ctf_amber_top.json")
function my_evaluator!(st::Common.State, do_forces::Bool)
    energy = Forcefield.Amber.evaluate!(topology, st, cut_off=1.2, do_forces=do_forces)
    return energy
end

my_evaluator! (generic function with 1 method)

## 6. Define the Driver

At this point, the system is ready to run. 

In [9]:
mc_driver = Drivers.MonteCarlo.MonteCarloDriver(my_sampler!, my_evaluator!, temperature=500.0, n_steps=1000)

MonteCarloDriver(sampler=my_sampler! evaluator=my_evaluator!, temperature=500.0, n_steps=1000)

## 7. (Optional) Define the Callbacks

Altough the system is ready to run, no output is being printed. In order to gain access to the inner working of the **Driver** callbacks need to be defined. More than one **CallbackObject**s can be defined, with independent call frequencies and functions. For this example, three callbacks will be prepared:
1. callback1 will be responsible to print the system status every 100 steps.
2. callback2 will print the current structure to a .xyz file every 100 steps.
3. callback3 makes use of the first `Vararg` made available by the MonteCarlo Driver (`acceptance_ratio`) to adjust the system's `step_size` so that, on average, 20% of the tried conformations are accepted.

In [10]:
callback1 = @Common.callback 100 function cb_status(step::Int64, st::Common.State, dr::Drivers.MonteCarlo.MonteCarloDriver, args...)
    write(stdout, @sprintf "(MC) Step: %4d | Energy: %.4ef | D step: %.3e | CS step: %.3e | AR: %.2f\n" step state.energy.eTotal dihedral_mutator.step_size crankshaft_mutator.step_size args[1])
end

CallbackObject(freq=100, callback=cb_status)

In [11]:
file_xyz = open("out/trajectory.pdb", "w")
callback2 = @Common.callback 100 function cb_print(step::Int64, st::Common.State, dr::Drivers.MonteCarlo.MonteCarloDriver, args...)
    Print.as_pdb(file_xyz, st, step = step)
end

CallbackObject(freq=100, callback=cb_print)

In [12]:
callback3 = @Common.callback 1 function cb_adjust_step_size(step::Int64, st::Common.State, dr::Drivers.MonteCarlo.MonteCarloDriver, args...)
    acceptance_ratio = 0.2
    if args[1] > acceptance_ratio
        dihedral_mutator.step_size * 1.05 < π ? dihedral_mutator.step_size *= 1.05 : dihedral_mutator.step_size = π
        crankshaft_mutator.step_size * 1.1 < π ? crankshaft_mutator.step_size *= 1.1 : crankshaft_mutator.step_size = π
    else
        dihedral_mutator.step_size * 0.95 > 1e-5 ? dihedral_mutator.step_size *= 0.95 : dihedral_mutator.step_size = 1e-5
        crankshaft_mutator.step_size * 0.9 > 1e-5 ? crankshaft_mutator.step_size *= 0.9 : crankshaft_mutator.step_size = 1e-5
    end
end

CallbackObject(freq=1, callback=cb_adjust_step_size)

## 8. Make science!

The set up of the system is complete, run the **Driver** and analyse the output!

In [13]:
Drivers.MonteCarlo.run!(state, mc_driver, callback1, callback2, callback3)
close(file_xyz)

(MC) Step:  100 | Energy: 6.5351e+05f | D step: 1.027e-03 | CS step: 8.132e-05 | AR: 0.17
(MC) Step:  200 | Energy: 2.8913e+05f | D step: 1.002e-04 | CS step: 1.000e-05 | AR: 0.18
(MC) Step:  300 | Energy: 1.2691e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.17
(MC) Step:  400 | Energy: 1.2813e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.16
(MC) Step:  500 | Energy: 1.1260e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.16
(MC) Step:  600 | Energy: 1.1280e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.16
(MC) Step:  700 | Energy: 1.1082e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.17
(MC) Step:  800 | Energy: 1.1213e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.17
(MC) Step:  900 | Energy: 1.0412e+05f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.17
(MC) Step: 1000 | Energy: 9.8113e+04f | D step: 1.000e-05 | CS step: 1.000e-05 | AR: 0.17
